In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import warnings
warnings.filterwarnings('ignore')
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


# Importing the dataset
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
combine= [df_train, df_test]
#print(df_train.columns.values)
#print(df_train.info())
#print('_' * 40)
#print(df_test.info())
#df_train.describe(include='O', exclude='category')



for dataset in combine:
    dataset.drop([col for col in dataset if dataset[col].count()/len(dataset) <= 0.3 ]+['PassengerId'], axis =1, inplace = True )
    
    
for dataset in combine:
    dataset.drop('Ticket', axis = 1, inplace = True)
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    
    
    
guess_ages = np.zeros((2,3))
guess_ages
for dataset in combine:
    for i in range(0,2):
        for j in range(0,3):
            guess_ages[i,j] = df_train[(df_train['Sex'] == i) & (df_train['Pclass'] == j+1)].dropna()['Age'].median()
    for i in range(0,2):
        for j in range(0,3):
            dataset.loc[(dataset.Age.isnull() & (dataset.Sex == i) & (dataset.Pclass == j+1)),'Age'] = guess_ages[i,j]
    dataset['Age'] = dataset['Age'].astype(int)
    
    
    
df_train.Embarked.value_counts().idxmax()
df_train.Embarked.fillna(df_train.Embarked.value_counts().idxmax(), inplace= True)
df_train.isna().sum()
from sklearn.impute import SimpleImputer

imr = SimpleImputer(missing_values= np.NaN, strategy='mean')
imr = imr.fit(df_test[['Fare']])
df_test['Fare'] = imr.transform(df_test[['Fare']]).ravel()

for dataset in combine: 
    dataset['FamilySize'] = dataset.SibSp + dataset.Parch + 1
    dataset.drop(columns=['SibSp','Parch'], inplace = True)
    
    
for dataset in combine: 
    dataset.Embarked = dataset.Embarked.map({'S':0,'Q':1,'C':2}).astype(int)
    
    
df_train['AgeBand'] = pd.cut(df_train.Age, bins= 5)
df_train.AgeBand.value_counts()


for dataset in combine: 
    dataset.loc[(dataset.Age <= 16) , 'Age'] = 0
    dataset.loc[(dataset.Age > 16) & (dataset.Age <= 32), 'Age'] = 1 
    dataset.loc[(dataset.Age > 32) & (dataset.Age <= 48), 'Age'] = 2 
    dataset.loc[(dataset.Age > 48) & (dataset.Age <= 64), 'Age'] = 3 
    dataset.loc[(dataset.Age > 64) & (dataset.Age <= 80), 'Age'] = 4 
df_train.drop(columns='AgeBand', inplace= True)
    
    
    
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset.FamilySize == 1,'IsAlone'] = 1
    dataset.drop(columns= 'FamilySize', inplace = True)
    
    
df_test.Fare.fillna(df_test.Fare.dropna().median(), inplace = True)



df_train['FareBand'] = pd.cut(df_train.Fare, bins = 4)

for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

df_train = df_train.drop(['FareBand'], axis=1)



df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand= False)
df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\.', expand= False)
df_train.drop('Name', axis = 1 , inplace = True)
df_test.drop('Name', axis = 1 , inplace = True )
combine = [df_train, df_test]



for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    

    
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [2])], remainder='passthrough')
new = df_train.columns.to_list()
new.remove('Sex')
new_col = ['Female', 'Male'] + new
df_train = pd.DataFrame(np.array(columnTransformer.fit_transform(df_train)), columns= new_col)




columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [3])], remainder='passthrough')
new = df_train.columns.to_list()
new.remove('Pclass')
new_col = ['First Class', 'Second Class', 'Third Class'] + new
df_train = pd.DataFrame(np.array(columnTransformer.fit_transform(df_train)), columns= new_col)


columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [8])], remainder='passthrough')
new = df_train.columns.to_list()
new.remove('Embarked')
new_col = ['Emb C', 'Emb Q', 'Emb S'] + new
df_train = pd.DataFrame(np.array(columnTransformer.fit_transform(df_train)), columns= new_col)



columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
new = df_test.columns.to_list()
new.remove('Sex')
new_col = ['Female', 'Male'] + new
df_test = pd.DataFrame(np.array(columnTransformer.fit_transform(df_test)), columns= new_col)


columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [2])], remainder='passthrough')
new = df_test.columns.to_list()
new.remove('Pclass')
new_col = ['First Class', 'Second Class', 'Third Class'] + new
df_test = pd.DataFrame(np.array(columnTransformer.fit_transform(df_test)), columns= new_col)


columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [7])], remainder='passthrough')
new = df_test.columns.to_list()
new.remove('Embarked')
new_col = ['Emb C', 'Emb Q', 'Emb S'] + new
df_test = pd.DataFrame(np.array(columnTransformer.fit_transform(df_test)), columns= new_col)



X_train = df_train.drop("Survived", axis=1)
Y_train = df_train["Survived"]
X_test  = df_test.copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 12), (891,), (418, 12))

# Model, Predict and Solve ! 

In [2]:
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
# note that the test set using the fitted scaler in train dataset to transform in the test set
#X_test = scaler.transform(X_test)

In [3]:
# data normalization with sklearn
#from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
#norm = MinMaxScaler().fit(X_train_scaled)

# transform training data
#X_train_norm = norm.transform(X_train_scaled)

# transform testing dataabs
#X_test_norm = norm.transform(X_test_scaled)

In [4]:
classifiers = []
model1 = xgboost.XGBClassifier(eval_metric='mlogloss')
classifiers.append(model1)


model2 = svm.SVC(C = 5)
classifiers.append(model2)


model3 = tree.DecisionTreeClassifier(criterion= 'entropy', splitter='best')
classifiers.append(model3)


model4 = RandomForestClassifier(criterion='entropy')
classifiers.append(model4)

model5 = LogisticRegression()
classifiers.append(model5)

model6 =KNeighborsClassifier(n_neighbors = 4)
classifiers.append(model6)


model7 = GaussianNB()
classifiers.append(model7)


model8 = Perceptron()
classifiers.append(model8)


model9 = LinearSVC()
classifiers.append(model9)

model10 = SGDClassifier()
classifiers.append(model10)

ModelName = [type(classifiers[i]).__name__ for i in range(0,10)]

Model_Score = []
for clf in classifiers:
    kfold = KFold(n_splits=10, random_state= 0, shuffle= True)
    Model_Score.append(cross_val_score(clf, X_train, Y_train, cv=kfold).mean()*100.0)
ModelDF = pd.DataFrame({'Model' : ModelName, 'Score' : pd.Series(Model_Score)})
ModelDF.sort_values(by = 'Score', ascending= False)

,Model,Score
1,SVC,81.594257
0,XGBClassifier,81.480649
4,LogisticRegression,80.700375
3,RandomForestClassifier,80.581773
2,DecisionTreeClassifier,80.471910
8,LinearSVC,79.352060
5,KNeighborsClassifier,79.235955
6,GaussianNB,77.776529
9,SGDClassifier,73.176030
7,Perceptron,72.945069


In [5]:
clf = LinearSVC()
clf.fit(X_train, Y_train)
Y_pred= clf.predict(X_test)
Y_pred
Y_pred = Y_pred.astype('int')
submit = pd.read_csv('gender_submission.csv')
submit['Survived'] = pd.Series(Y_pred)
submit.to_csv('KaggleTitanicSubmission.csv', index=False)